# Transcribe Recordings
paid service [Goodtape](http://goodtape.io) will be used for the transcriptions

## Imports

In [1]:
%reload_ext autoreload
%autoreload 2

In [76]:
import requests
import pandas as pd
import os
import shutil
import numpy as np
import io
import json
import math

from requests.utils import quote

# audio
from pydub import AudioSegment

# local imports
from lib_henryk.logger import *
from lib_henryk.config import *

from lib_henryk.transcription import transcription

## Params

In [3]:
# load api keys
from dotenv import load_dotenv
_ = load_dotenv()

WEBHOOKS_TOKEN_ID = 'd1d3576a-615d-418d-bb5b-2581e23c94dc'
GOODTAPE_API_KEY = os.getenv("GOODTAPE_API_KEY")

## Load Dataset

In [4]:
df = pd.read_csv(FILE_AUDIO_STATS_CSV)
df_transcription = df.copy()
df_transcription['transcription_id'] = ''

## Run Transcriptions

In [99]:
# load sync dataframe
if os.path.exists(FILE_TRANSCRIPTION_CSV):
    print(f'transcription log exists, reading it from {FILE_TRANSCRIPTION_CSV}')
    df_transcription = pd.read_csv(FILE_TRANSCRIPTION_CSV)

transcription log exists, reading it from ../../data/processed/henryk_audio_transcription.csv


In [102]:
# perform transcriptions
transcription.submit_transcriptions_goodtape(
    df=df_transcription, 
    webhooks_token_id=WEBHOOKS_TOKEN_ID,
    goodtape_api_key=GOODTAPE_API_KEY,
    path_recordings=DIR_WIADOMOSCI_DO_HENRYCZKA,
    start=0, stop=300,
    verbose=False
)

processing [287]: Henryk 2023-06-16 Tatuś opowiada o tym jak warto popełniać błędy a Smoki budują mur.m4a
processing [288]: Henryk 2023-06-17 Tatuś opowiada o malowaniu bawialni a Żaba śni że jest warzywem.m4a
processing [289]: Henryk 2023-06-19 Tata opowiada o piasku i polskim morzu a Żaba prosi smoki by ją obudziły.m4a
processing [290]: Henryk 2023-06-20 Tata opowiada o kierunkach geograficznych a Żaba budzi się w końcu.m4a
processing [291]: Henryk 2023-06-22 Tata opowiada o mapie a Smoki urządzają pranie.m4a
processing [292]: Henryk 2023-06-25 Tatuś opowiada o Babusi i Dziadziusiu a Makrauchenia rozpoczęła opowieść o Din...
processing [293]: Henryk 2023-06-27 Tatuś opowiada o cioci Monice a Makrauchenia opowiada o Dinozaurach i o tym że i...
processing [294]: Henryk 2023-06-29 Tata opowiada o prezencie od Henryczka a Rysio Kapustnik zapowiada smokom Niespodz...
processing [295]: Henryk 2023-06-30 Tatuś opowiada o rozmowie z ciocią Sylwią a Smoki poznają nowe

## Fetch Transcriptions JSONs

In [ ]:
# need to wait until transcriptions are available
exit_cell()

In [104]:
# retrieve goodtape server responses from webhooks
transcription.retrieve_responses_goodtape_via_webhooks(
    df=df_transcription,
    webhooks_token_id=WEBHOOKS_TOKEN_ID, 
    path_transcriptions_json=DIR_TRANSCRIPTIONS_JSON
)

# process json to text
transcription.process_transcriptions_json_to_text(
    df=df_transcription, 
    path_transcriptions_json=DIR_TRANSCRIPTIONS_JSON, 
    path_transcriptions_txt=DIR_TRANSCRIPTIONS_TXT,
    verbose=False
)

found 33 requests to fetch
retrieved transcription [0]: 22a93a64-ce8f-4ce7-a60a-23f8d37337a4 -> file [298]: Henryk 2023-07-04 Tatuś robi kwietniki do bawialni a Żaba słuc...
retrieved transcription [1]: 92c6bedd-a41f-4485-8e7f-2887f5e6b6d3 -> file [295]: Henryk 2023-06-30 Tatuś opowiada o rozmowie z ciocią Sylwią a...
retrieved transcription [2]: 66d74140-4412-4696-8019-d34496601f44 -> file [297]: Rodzice Henryk 2023-07-02 Dziadziuś i Babusia opowiadają o kempi...
retrieved transcription [3]: 5f9b87b9-4081-4882-ad4c-fe6a2810e5c2 -> file [296]: Henryk 2023-07-01 Tatuś spotkał się z wujkiem Michałem przy og...
retrieved transcription [4]: ab7ec2e6-4848-4e8b-874f-b13474027f92 -> file [294]: Henryk 2023-06-29 Tata opowiada o prezencie od Henryczka a Rysio...
retrieved transcription [5]: 25a6d229-9888-47fd-80e3-565f663f0da3 -> file [292]: Henryk 2023-06-25 Tatuś opowiada o Babusi i Dziadziusiu a Makra...
retrieved transcription [6]: bef3748e-38f6-455b-ac76-d59a38d4ee94 -> file [299]

## Cleanup missing requests
- transcription ids for the requests that were lost will be removed, only those that were processed will remain

In [109]:
# cleanup stale transcription ids
transcription.cleanup_missing_transcriptions(
    df=df_transcription, 
    path_transcriptions_json=DIR_TRANSCRIPTIONS_JSON, 
    path_transcriptions_txt=DIR_TRANSCRIPTIONS_TXT
)

# save transcription log
df_transcription.to_csv(FILE_TRANSCRIPTION_CSV, index=False)

there are 300 processed and valid transcriptions


In [ ]:
# df_transcription = pd.read_csv(FILE_TRANSCRIPTION_CSV)